In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

social = pd.read_csv("Aggregated_reddit_twitter.csv")
social = social.rename(columns={"date": "timestamp"})
social["timestamp"] = pd.to_datetime(social["timestamp"])

In [ ]:
social = social[social.content != "u/circular360"]
social = social[social.content != "There you go"]
social = social[
    social.content != "Why force the other player to stay around? Just give the winning player the option to continue the game with a perma passing AI controlling the surrendering player."]
# crypto = pd.read_csv("transformed_crypto.csv")
# crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)
crypto = pd.read_csv("crypto_differences.csv")
crypto["timestamp"] = pd.to_datetime(crypto["timestamp"]).dt.tz_localize(None)

social = social.sort_values(by="timestamp")
# social.dtypes
sr = social.resample("T", on="timestamp").size()
freq = sr.reset_index()

freq = freq.rename(columns={0: "n_comments"})

In [ ]:
freq["hour"] = freq["timestamp"].dt.hour
freq["day"] = freq["timestamp"].dt.dayofweek
freq["month"] = freq["timestamp"].dt.month
freq

In [ ]:
alp = pd.read_csv("df_alphavantage_scores.csv")
alp["timestamp"] = pd.to_datetime(alp["timestamp"])
alp = alp.drop("Unnamed: 0", axis=1)
frqn = pd.merge(freq, alp, on="timestamp", how="left")

In [ ]:
mg = pd.merge(crypto, frqn, on="timestamp", how="inner")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset


In [ ]:
shortmg = mg.loc[0:20000]
shortmg.dropna()


In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)


class MyTimeSeriesDataset(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data.loc[index], self.y_data[index]

    def __len__(self):
        return len(self.X_data)

# data = MyTimeSeriesDataset(X_data=shortmg[['n_comments', 'XRP-PERP', 'ETH-PERP']], y_data=shortmg['BTC-PERP'])

In [10]:
# Load your dataset into a pandas DataFrame called 'data'
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Scale the features using MinMaxScaler
cs = ['BTC-PERP', 'XRP-PERP', 'ETH-PERP', 'SOL-PERP', 'DOGE-PERP']
scaled_data = shortmg[cs + ["hour", "day", "month", "n_comments"]]
# scaled_data = shortmg[['XRP-PERP', 'ETH-PERP', 'BTC-PERP', 'SOL-PERP', 'DOGE-PERP', 'n_comments']].to_numpy()

# scaled_data = abs(scaled_data)


# scaled_data = scaled_data.to_numpy()
#
scaler = StandardScaler()
#
scaled_data = scaled_data.to_numpy()
# scaled_data = scaler.fit_transform(scaled_data)


# Create sequences and targets for the RNN model
seq_length = 60
X_data = []
y_data = []

future = 5

for i in range(len(scaled_data) - seq_length - future):
    X_data.append(scaled_data[i:i + seq_length].flatten())
    y_data.append(scaled_data[i + seq_length + future])

x_data = scaler.fit_transform(X_data)
# y_data = np.delete(y_data, [-1, -2, -3, -4], axis=1)
y_data = np.delete(y_data, [-1, -2, -3, -4, -5, -6, -7, -8], axis=1)
# to make classifier
epsilon = 0.002


def f(x):
    if x > epsilon:
        return 1
    elif x < -epsilon:
        return 2
    else:
        return 0


[0, 0, 1]
[0, 1, 0]
[1, 0, 0]
y_data = list(map(f, y_data))

print(y_data[1:10])
X_data, y_data = np.array(X_data), np.array(y_data)

y_data = y_data.astype(np.int64)


[2, 1, 2, 1, 2, 2, 2, 2, 2]


In [11]:
train_size = int(len(X_data) * 0.8)
X_train, X_test = X_data[:train_size], X_data[train_size:]
y_train, y_test = y_data[:train_size], y_data[train_size:]

In [12]:
m = y_test.mean(axis=0)
((y_test - m) ** 2).mean()
y_test.var(), y_train.var()

(0.6888305337275618, 0.6369096698707718)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# regressor = RandomForestRegressor(random_state=0, max_depth=5, n_estimators=20)
# regressor = SVR()
regressor = LogisticRegression()
regressor.fit(X_train, y_train)
pred = regressor.predict(X_test)


/home/dario/PycharmProjects/BAINSACryptocurrencies/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
from torch.utils.data import DataLoader

batch_size = 256
train_dataset = TimeSeriesDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
test_dataset = TimeSeriesDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float())

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [15]:
# classifier
# y_data = y_data.flatten()
# y_data.shape

In [16]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNNModel, self).__init__()

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])
        return out


class TransformerModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_layers, output_size):
        super(TransformerModel, self).__init__()

        self.embedding = nn.Linear(input_size, d_model)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers)
        self.fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x = x.transpose(0, 1)
        x = self.transformer_encoder(x)
        x = x.transpose(0, 1)
        x = self.fc(x[:, -1, :])
        return x


class DenseModel(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 256)
        self.ff = nn.Sequential(
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_size)
        )

    def forward(self, x):
        # x = torch.flatten(x, 1)
        x = self.lin(x)
        x = self.ff(x)
        o = f.log_softmax(x, dim=1)
        return o

In [17]:
from torch import optim
import torch.nn.functional as f

input_size = X_data.shape[1]
# hidden_size = 256
# num_layers = 8
# output_size = y_data.shape[1]
output_size = 3
# output_size = 1

# model = RNNModel(input_size, hidden_size, num_layers, output_size)
# model = TransformerModel(input_size, d_model=32, nhead=2, num_layers=2, output_size=output_size)
model = DenseModel(input_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
input_size, output_size

(540, 3)

In [18]:
import platform

num_epochs = 30
log = False
#
if "Linux" in platform.platform():
    model = torch.compile(model)
#
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
        y_pred = model(X_batch)
        #
        optimizer.zero_grad()
        y_batch = y_batch.type(torch.LongTensor)
        loss = f.nll_loss(y_pred, y_batch)

        loss.backward()
        optimizer.step()
        # optimizer.step()

        if batch_idx % 20 == 0 and True:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(X_batch), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss))

    model.eval()
    with torch.no_grad():
        correct = 0
        test_loss = 0
        for X_batch, y_batch in test_loader:
            y_pred = model(X_batch)
            y_batch = y_batch.type(torch.LongTensor)

            loss = f.nll_loss(y_pred, y_batch)
            test_loss += loss

            pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(y_batch.view_as(pred)).sum().item()

        test_loss = test_loss / len(test_loader)

        print(f"Epoch: {epoch + 1}, Train Loss: {loss.item():.4f}, Test Loss: {test_loss:.4f}, accuracy: {100*correct/len(test_loader.dataset)}")



tensor(-0.3326, grad_fn=<NllLossBackward0>)
-0.33257535099983215


Process ForkProcess-8:
Process ForkProcess-7:
Process ForkProcess-3:
Process ForkProcess-2:
Process ForkProcess-5:
Process ForkProcess-4:
Process ForkProcess-6:
Process ForkProcess-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.10/mult

KeyboardInterrupt: Interrupted by user